In [ ]:
import pandas as pd
import requests
import numpy as np
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes
from bs4 import BeautifulSoup

Solving environment: done

# All requested packages already installed.

Solving environment: | 

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [ ]:
response = requests.get(url)
soup = BeautifulSoup(response.text,'lxml')
table = soup.find_all('table')[0] 
column_headers = [th.getText() for th in 
                  soup.findAll('tr', limit=1)[0].findAll('th')]
column_headers[-1] = column_headers[-1].strip()
data_rows = table.findAll('tr')[1:]
row_data = [[td.getText() for td in data_rows[i].findAll('td')]
            for i in range(len(data_rows))]
df = pd.DataFrame(row_data, columns=column_headers)
df.Neighbourhood = df.Neighbourhood.map(lambda x: str(x)[:-1])

In [ ]:
df1 = df[df.Borough != 'Not assigned']
df1['Neighbourhood'] = np.where(df1.Neighbourhood == 'Not assigned', df1.Borough, df1.Neighbourhood)
df1.reset_index(inplace=True)
df1 = df1.drop(['index'], axis=1)

In [ ]:
df2 = df1.groupby(['Postcode', 'Borough'], sort=False)['Neighbourhood'].apply(lambda x: ', '.join(x))
df3 = pd.DataFrame(df2).reset_index(inplace=False)
df3[0:5]

In [ ]:
df3.shape